# GOLD: Stance Matchup Matrix (Materialized)

Builds and refreshes a materialized stance matchup Delta table and view from silver data.


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Widgets/catalog
try:
    dbutils.widgets.text("storage_account", "storagetmufc")
    dbutils.widgets.text("secret_scope", "kv-scope")
    dbutils.widgets.text("key_name", "adls-account-key")
    dbutils.widgets.text("silver_db", "ufc_silver")
    dbutils.widgets.text("gold_db", "ufc_gold")
except Exception:
    pass

silver_db = dbutils.widgets.get("silver_db") if 'dbutils' in globals() else "ufc_silver"
gold_db = dbutils.widgets.get("gold_db") if 'dbutils' in globals() else "ufc_gold"

try:
    storage_account = dbutils.widgets.get("storage_account")
    secret_scope = dbutils.widgets.get("secret_scope")
    key_name = dbutils.widgets.get("key_name")
    account_key = dbutils.secrets.get(secret_scope, key_name)
    spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", account_key)
except Exception:
    pass

try:
    spark.sql("USE CATALOG hive_metastore")
except Exception:
    try:
        spark.catalog.setCurrentCatalog("hive_metastore")
    except Exception:
        pass
spark.sql(f"CREATE DATABASE IF NOT EXISTS {silver_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")
try:
    spark.catalog.setCurrentDatabase(gold_db)
except Exception:
    spark.sql(f"USE DATABASE {gold_db}")
print("Silver DB:", silver_db, "| Gold DB:", gold_db)


In [ ]:
# Load silver
fights = spark.table(f"hive_metastore.{silver_db}.espn_fights_silver")
ath = spark.table(f"hive_metastore.{silver_db}.espn_athletes_silver")

# Normalize stance
norm = F.udf(lambda s: (s or "").strip().lower().replace(" ", "_"))
ath1 = (ath
    .withColumn("stance_norm", F.when(F.col("stance").isNull() | (F.trim("stance")==""), F.lit("unknown")).otherwise(norm("stance")))
    .select("athlete_id","stance_norm","reach_cm","height_cm","country"))

# Build athlete-fight rows (two rows per fight)
left_rows = (fights
    .select("competition_id","event_id","event_date","event_year","weight_class",
            F.col("fighter_a_name").alias("athlete_name"),
            F.col("fighter_a_winner").alias("is_winner"))
    .withColumn("side", F.lit("A")))
right_rows = (fights
    .select("competition_id","event_id","event_date","event_year","weight_class",
            F.col("fighter_b_name").alias("athlete_name"),
            F.col("fighter_b_winner").alias("is_winner"))
    .withColumn("side", F.lit("B")))

af = left_rows.unionByName(right_rows)

# Fuzzy join athlete_name to athletes by display/full name in bronze silver is not guaranteed; we use name match
# Note: If you have athlete_id in bronze fights later, switch to exact join by athlete_id
ath_name_cols = ["full_name","display_name"]
ath_names = (spark.table(f"hive_metastore.{silver_db}.espn_athletes_silver")
             .select(F.col("full_name").alias("name1"), F.col("display_name").alias("name2"), "athlete_id"))

afj = (af
    .join(ath_names, (F.lower(F.col("athlete_name"))==F.lower(F.col("name1"))) | (F.lower(F.col("athlete_name"))==F.lower(F.col("name2"))), "left")
    .join(ath1, "athlete_id", "left"))

# Self-join to get opponent stance
w = Window.partitionBy("competition_id").orderBy(F.lit(1))
with_op = (afj.alias("x")
    .join(afj.alias("y"), (F.col("x.competition_id")==F.col("y.competition_id")) & (F.col("x.side")!=F.col("y.side")), "inner")
    .select(
        F.col("x.competition_id").alias("competition_id"),
        F.col("x.event_id").alias("event_id"),
        F.col("x.event_date").alias("event_date"),
        F.col("x.event_year").alias("event_year"),
        F.col("x.weight_class").alias("weight_class"),
        F.col("x.athlete_id").alias("athlete_id"),
        F.col("x.stance_norm").alias("stance"),
        F.col("y.stance_norm").alias("opp_stance"),
        F.col("x.is_winner").alias("is_winner")
    ))

# Aggregate stance vs stance win counts
agg = (with_op
    .groupBy("weight_class","stance","opp_stance")
    .agg(F.count("*").alias("fights"), F.sum(F.when(F.col("is_winner") == True, 1).otherwise(0)).alias("wins"))
    .withColumn("win_rate", F.when(F.col("fights")>0, F.col("wins")/F.col("fights")).otherwise(F.lit(0.0)))
)

# Materialize Delta + View
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")
agg.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(f"hive_metastore.{gold_db}.mv_stance_matchups")
spark.sql(f"CREATE OR REPLACE VIEW hive_metastore.{gold_db}.v_stance_matchups AS SELECT * FROM hive_metastore.{gold_db}.mv_stance_matchups")

# Preview
display(spark.table(f"hive_metastore.{gold_db}.mv_stance_matchups").orderBy(F.desc("fights")).limit(50))
